# Import module

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import sqlite3
import json

In [2]:
# 載入自訂義檔案, 地址轉經緯度function
from Function_list import get_latitude_longitude

# Cinema infomation webcraw

In [3]:
def dataset_webcraw(url):
    response = requests.get(url)
    res = BeautifulSoup(response.text) # 預設html.parser，不打也相同
    latest_data = res.find_all('div', class_ = 'download-item')[-1].a['href']
    return latest_data
url = 'https://data.gov.tw/dataset/22213'
latest_data = dataset_webcraw(url)

# Data Clean & Feature

In [ ]:
df = pd.read_csv(latest_data)
cinema_data = df[['事業名稱','地址']]
cinema_data.dropna(inplace=True)

In [ ]:
# 臺 轉 台
cinema_data['地址'] = cinema_data['地址'].apply(lambda x: '台' + x[1:] if x.startswith('臺') else x)

# 縣市
cinema_data['縣市'] = cinema_data['地址'].map(lambda x:x[:3])

# 換順序
cinema_data = cinema_data.reindex(columns = ['縣市', '事業名稱', '地址'])

## 地址錯誤手動修正


In [7]:
# 時代戲院	台中市清水區中山路339號3F~5F
cinema_data.loc[cinema_data['事業名稱'] == '時代戲院', '地址'] = '台中市清水區中山路339號3F~5F'

In [8]:
# 新增經緯度，按下去就會刷google API 哦!
cinema_data['經緯度'] = cinema_data['地址'].map(get_latitude_longitude)

In [9]:
# 經緯度turple轉string
cinema_data['經緯度'] = cinema_data['經緯度'].map(json.dumps)

# Save to DataBase、JSON

In [10]:
# 不帶index 轉DB
conn = sqlite3.connect('TW_cinema_info.db')
cinema_data.to_sql('TW_cinema_info_latest', con=conn, if_exists='replace', index=False)
conn.close()

In [13]:
# # 不index 轉JSON
# # 先轉縣市當key、其他為values的dict
# # 在字典轉JSON
# grouped_county = cinema_data.groupby('縣市').apply(lambda x: x.drop('縣市', axis=1).to_dict(orient='records')).to_dict()
# with open('TW_cinema_info_latest.json', 'w', encoding='utf-8') as file:
#     json.dump(grouped_county, file, ensure_ascii=False)

# 載入DB、JSON

In [11]:
# DB
conn = sqlite3.connect('TW_cinema_info.db')
data_df = pd.read_sql_query("SELECT * FROM TW_cinema_info_latest", conn)
conn.close()

In [14]:
# # JSON
# with open('TW_cinema_info_latest.json', 'r', encoding='utf-8') as f:
#     data_dict = json.load(f)

# Check

In [12]:
# 經緯度欄位是字串，可透過eval()轉list，方便取出經度緯度
data_df

,縣市,事業名稱,地址,經緯度
0,台北市,國賓大戲院,台北市成都路88號,"[25.042998, 121.50454]"
1,台北市,台北新光影城,台北市萬華區西寧南路36號4樓,"[25.0452169, 121.506424]"
2,台北市,樂聲大戲院,台北市萬華區武昌街2段85號1~5樓,"[25.0452854, 121.5048364]"
3,台北市,今日秀泰影城,台北市峨嵋街52號4.6.8樓,"[25.0329636, 121.5654268]"
4,台北市,喜滿客絕色電影城,台北市萬華區漢中街52號8至11樓,"[25.0438973, 121.5071127]"
...,...,...,...,...
110,花蓮縣,花蓮新天堂樂園威秀影城,花蓮縣吉安鄉南濱路一段503號3-4樓,"[23.9383814, 121.595437]"
111,台東縣,台東秀泰影城,台東縣台東市新生路93號3樓,"[22.7523536, 121.147811]"
112,金門縣,金獅影城,金門縣金湖鎮中山路8-5號西棟3樓,"[24.4417138, 118.3630416]"
113,金門縣,國賓影城@金門昇恆昌金湖廣場,金門縣金湖鎮太湖路二段198號6樓,"[24.4392162, 118.4186129]"
